# AWS Glue Job - Load Forex data to Bronze Stage from API source


## Set the Glue session parameters

In [16]:
%iam_role arn:aws:iam::212430227630:role/LabRole
%region us-east-1
%number_of_workers 2

%idle_timeout 30
%glue_version 4.0
%worker_type G.1X

%%configure 
{
  "--enable-metrics": "true",
  "--enable-observability-metrics": "true"
}

Current iam_role is arn:aws:iam::212430227630:role/LabRole
iam_role has been set to arn:aws:iam::212430227630:role/LabRole.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: 2
Setting new number of workers to: 2
Current idle_timeout is 30 minutes.
idle_timeout has been set to 30 minutes.
Setting Glue version to: 4.0
Previous worker type: G.1X
Setting new worker type to: G.1X
The following configurations have been updated: {'--enable-metrics': 'true', '--enable-observability-metrics': 'true'}


In [19]:
%extra_py_files s3://cryptoengineer/gluejobs-py-modules/load.py, s3://cryptoengineer/gluejobs-py-modules/storage.py
%additional_python_modules yfinance

Extra py files to be included:
s3://cryptoengineer/gluejobs-py-modules/load.py
s3://cryptoengineer/gluejobs-py-modules/storage.py
Additional python modules to be included:
yfinance


##  Set up and start your interactive session.


In [22]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import boto3

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 30
Session ID: b7d67d73-ea8e-4a4a-865d-177799d3e940
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--enable-metrics true
--enable-observability-metrics true
--extra-py-files s3://cryptoengineer/gluejobs-py-modules/load.py,s3://cryptoengineer/gluejobs-py-modules/storage.py
--additional-python-modules yfinance
Waiting for session b7d67d73-ea8e-4a4a-865d-177799d3e940 to get into ready status...
Session b7d67d73-ea8e-4a4a-865d-177799d3e940 has been created.



## Batch Load - FOREX

### Load the libraries and dependencies

In [2]:
from datetime import datetime, timedelta, timezone
import pyspark.sql.functions as F

import pandas as pd
import load

### Set AWS Storage parameters


In [4]:
BUCKET_NAME = "cryptoengineer"
PREFIX = "datalake/bronze/forex"

### Load job parameters

In [8]:
glue_client = boto3.client("glue")
# Check if params comde from a GLUE workflow
if '--WORKFLOW_NAME' in sys.argv and '--WORKFLOW_RUN_ID' in sys.argv:
    print("Running in Glue Workflow")
    
    glue_args = getResolvedOptions(
        sys.argv, ['WORKFLOW_NAME', 'WORKFLOW_RUN_ID']
    )
    
    print("Reading the workflow parameters")
    workflow_args = glue_client.get_workflow_run_properties(
        Name=glue_args['WORKFLOW_NAME'], RunId=glue_args['WORKFLOW_RUN_ID']
    )["RunProperties"]

    
    base= workflow_args['base']
    time_frame = int(workflow_args['time_frame'])
    symbols = workflow_args['symbols']
    api_key = workflow_args['api_key']

else:
    # Check if params comde from a Glue Job
    try:
        args = getResolvedOptions(sys.argv,
                                  ['JOB_NAME',
                                   'base',
                                   'time_frame',
                                   'symbols',
                                   'api_key'])
        base= args['base']
        time_frame = int(args['time_frame'])
        symbols = args['symbols']
        api_key = args['api_key']
        print("Running as Job")        
    except:
        # Set the params for an interactive session
        print("Running as an interactive session")
        base= 'USD'
        time_frame = 384
        symbols = "USDEUR"
        api_key= ""
        


Running as an interactive session


In [9]:
print("base: ", base)
print("Time Frame: ", time_frame)
print("Symbols: ", symbols)
print("API Key: ", api_key)

base:  USD
Time Frame:  384
Symbols:  USDEUR
API Key:  xGFdE9Ydrcr1oCDJiCjHiZnkqUnQnjaH


#### Set the start and end dates for the data you want to load

In [10]:
# Start date
start_date = (datetime.utcnow() - timedelta(hours=time_frame)).strftime("%Y-%m-%d")
end_date = datetime.utcnow().strftime("%Y-%m-%d")

print("Start date; ",start_date," End date: ",end_date)

Start date;  2024-09-07  End date:  2024-09-23


## Read max date loaded from INFO table

We read the last date source data were loaded to remove duplicates rows

In [11]:
PREFIX_INFO_TABLE='datalake/gold/forex'
path=f"s3://{BUCKET_NAME}/{PREFIX_INFO_TABLE}"
print("Path:", path)

Path: s3://cryptoengineer/datalake/gold/forex


In [12]:
df_max_dates = (
    spark
    .read
    .parquet(path)
    .filter(F.col("stage")=='bronze')
    .select('symbol','base_currency','frequencies','end_datetime')
    .toPandas()
)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [13]:
df_max_dates.head(10)

   symbol base_currency frequencies        end_datetime
0  USDCHF           USD       15min 2024-09-06 16:45:00
1  USDEUR           USD       15min 2024-09-06 16:45:00
2  USDGBP           USD       15min 2024-09-06 16:45:00
3  USDJPY           USD       15min 2024-09-06 16:45:00


In [17]:
df_max_dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   symbol         4 non-null      object        
 1   base_currency  4 non-null      object        
 2   frequencies    4 non-null      object        
 3   end_datetime   4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 256.0+ bytes


## Load the historical rates - 15min frequency

Set some fixed values we use to populate the bronze stage

In [14]:
freq='15min'
source='FMP'

Now, we iterate the symbols list and load the data for each symbol from the source, remove data rows already loaded and save to a Pandas dataframe.

In [20]:
df= pd.DataFrame()
for symbol in symbols.split(","):
    print("Loading: ", symbol)
    symbol_df = load.load_batch_freq_rates(base=base,
                                          start_date=start_date,
                                          end_date=end_date,
                                          freq=freq,
                                          symbol=symbol,
                                          api_key=api_key,
                                          source=source
    )
    print("Records loaded: ", len(symbol_df))
    # Get the max date time already loaded 
    max_datetime=df_max_dates[(df_max_dates['symbol'] ==symbol) & (df_max_dates['base_currency'] ==base) 
                              & (df_max_dates['frequencies'] ==freq)]['end_datetime'].max()
    # Extract only the new data
    symbol_df = symbol_df[pd.to_datetime(symbol_df['date']) > max_datetime]
    print("New records: ", len(symbol_df))
    
    # Complete the table schema 
    if len(symbol_df)>0:
        symbol_df = load.set_schema_table(symbol_df, symbol, source, freq, base)
        print("Records: ", len(symbol_df))
        df = pd.concat([df, symbol_df])
    else:
        print("No data for: ", symbol)


Loading:  USDEUR
Leidos  997
Creating the dataframe
Records loaded:  997
New records:  997
Records:  997


In [21]:
print("Records: ", len(df))

Records:  997


In [22]:
df.head(5)

              datetime     open  ...   load_date   type
0  2024-09-23 05:00:00  0.90147  ...  2024-09-23  FOREX
1  2024-09-23 04:45:00  0.90115  ...  2024-09-23  FOREX
2  2024-09-23 04:30:00  0.90157  ...  2024-09-23  FOREX
3  2024-09-23 04:15:00  0.90188  ...  2024-09-23  FOREX
4  2024-09-23 04:00:00  0.90088  ...  2024-09-23  FOREX

[5 rows x 18 columns]


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 996
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       997 non-null    object             
 1   open           997 non-null    float64            
 2   low            997 non-null    float64            
 3   high           997 non-null    float64            
 4   close          997 non-null    float64            
 5   volume         997 non-null    int64              
 6   year           997 non-null    object             
 7   month          997 non-null    object             
 8   day            997 non-null    object             
 9   time           997 non-null    object             
 10  date           997 non-null    object             
 11  base_currency  997 non-null    object             
 12  source         997 non-null    object             
 13  frequency      997 non-null    object             

## Append the batch data to RAW table

Set the destination raw table

In [24]:
path=f"s3://{BUCKET_NAME}/{PREFIX}"
print("Path:",path)

Path: s3://cryptoengineer/datalake/bronze/forex


Save data to datalake, bronze stage, in parquet format and partitioned by load_date

In [25]:
if len(df)>0:
    print("Saving data to: ", path)
    (
        spark.createDataFrame(df)
        .repartition("load_date")
        .write
        .format("parquet")
        .mode("append")
        .partitionBy(['load_date'])
        .save(path)
    )

Saving data to:  s3://cryptoengineer/datalake/bronze/forex
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
